# Create R Script Tools

Another main use of the bridge is to wrap R functionality into a geoprocessing script so it can be called as a tool in ArcGIS. This is useful when you have functions in R that you use repeatedly and want to visualize the results on a map, to continue your analysis in ArcGIS, or to share with others. R script tools can be fully integrated into ArcGIS and seamlessly linked with other Geoprocessing tools, or Python script tools to create thorough and powerful workflows. This makes it easier to share your workflows with other members of your organization, especially with those that are not familiar with the R language. 

See below for the various topics related to R script tool creation:

 - [arcgisbinding Script Tool Functionality](#arcgisbinding-Script-Tool-Functionality)
  - [arc.env Functionality](#arc.env-Functionality)    
  - [arc.progress_label and arc.progress_pos Functionality](#arc.progress_label-and-arc.progress_pos-Functionality)
 - [Components of an R Script Tool](#Components-of-an-R-Script-Tool)
  - [Required R Packages](#Required-R-Packages)
  - [Define Inputs and Outputs](#Define-Inputs-and-Outputs)
  - [Open and Read-in Data](#Open-and-Read-in-Data)
  - [Custom R Functionality](#Custom-R-Functionality)
  - [Generate Results and Messages](#Generate-Results-and-Messages)
  - [Example R Script Tools](#Example-R-Script-Tools)
 - [Components of ArcGIS Script Tool UI Creation](#Components-of-ArcGIS-Script-Tool-UI-Creation)
 - [Hands-on Practice: Script Tool Creation](#Hands-on-Practice:-Script-Tool-Creation)
  - [Exercise: Create an R script geoprocessing tool](#Exercise:-Create-an-R-script-geoprocessing-tool)
  

## arcgisbinding Script Tool Functionality

The **arcgisbinding** package contains several functions to help you design and customize the new script tools you create:

 * `arc.env()`
 
 * `arc.progress_label()`
 
 * `arc.progress_pos()`
 

### arc.env Functionality
 
 When working in ArcGIS, a user has the ability to customize their project environment using geoprocessing environment settings. This includes things such as setting an output coordinate system, defining a processing extent, setting a random number seed, etc. If a user has preset some of their environment variables, this may impact the results of your R script tool. The `arc.env()` function enables you to get the local ArcGIS geoprocessing tool environment settings and to check if they are set appropriately. 

When used in the sample script tool below, the geoprocessing tool environment settings are stored in the `env` variable by using the `arc.env()` function. This allows for you to check a user's settings as needed within your script. While you are unable to override a user's local settings, you can throw a warning or error message to alert the user that they need to adjust their environments before continuing with your tool if a certain setting impacts your tool's results. 

For example, below we check to see what the package workspace path is set to. 

```R
tool_exec <- function(in_params, out_params)
 {
  env = arc.env()
  wkspath <- env$workspace
  .
  .
  .
 }
```

An overview of all the different environment settings possible can be found [here](https://pro.arcgis.com/en/pro-app/tool-reference/environment-settings/an-overview-of-geoprocessing-environment-settings.htm). 
 
### arc.progress_label and arc.progress_pos Functionality
 
 The **arcgisbinding** package also allows you to customize the tool run experience users have with your script tool through use of the `arc.progress_label()` and `arc.progress_pos()` functions.
 
> Note: Currently, these functions only work with ArcGIS Pro. 

When running a geoprocessing tool in ArcGIS, a tool run status bar appears that can display custom messages to indicate the status of the run. In addition, the tool run progress bar can be customized to appear at different levels of completion based on the current stage of the run. 

With the `arc.progress_label()` function, you can customize what messages your users see based on where the script is at while they are running your tool. While the tool run status bar typically continuously moves back and forth by default, if you wish, you can customize it to appear at a certain percentage of completion by entering a value from 0 to 100 to represent the current percentage. Both of these functions are used multiple times in the sample script tool below to demonstrate how they might be used when creating a new geoprocessing script tool. 

Here are some examples excerpted from a script tool:

```R
tool_exec <- function(in_params, out_params)
 {
  arc.progress_label("Loading Dataset...")
  arc.progress_pos(25)
  .
  .
  .
  arc.progress_label("Obtaining Attribute Names...")
  arc.progress_pos(50)
  .
  .
  .
  arc.progress_label("Obtaining Geometry Information...")
  arc.progress_pos(75)
  .
  .
  .
  arc.progress_label("Writing result dataset...")
  arc.progress_pos(100)
 }
```
 

## Components of an R Script Tool

While the **arcgisbinding** package offers a high level of customization over the resulting script tools that are produced from wrapped R functionality, each script, no matter the analysis, will follow a similar framework and will contain certain elements. 

To begin, the `tool_exec()` function is the container each of these elements will fall within. 

```R
tool_exec <- function(in_params, out_params)
 {
  .
  .
  .
 }
```

Within the `tool_exec()` function, you can leverage **arcgisbinding** functions like `arc.env()` to check the geoprocessing environment settings of the Pro version users of your tool are working with. This enables you to check the status of settings that might impact the result of your tool. Addtionally, the `print()` function can be used to construct message window printouts for the users of your tool. 

```R
  env <- arc.env()
  workspace <- env$workspace
  
  print(workspace)
```

### Required R Packages

Users of your script tool will need to already have R installed on their local machines. Additionally, any R packages that your script requires, will need to be installed and loaded into their local R library. This latter step can be done directly by your script tool. By performing a check, missing packages and their dependencies can be installed through the `install.packages()` R function. All needed packages, can then be directly loaded into the user's R library so your tool can perform its analysis in ArcGIS without the user ever needing to open R. 

```R
  if(!requireNamespace("caret", quietly = TRUE))
    install.packages("caret", quiet = TRUE, dependencies = TRUE)
  
  require(caret)
```

### Define Inputs and Outputs

On the script side, the input and the output parameters of your tool must be specified so they can be used in your script. These are referenced based on the order you want them to appear in your script tool's UI. The first parameter box has an index value of 1. The second parameter box has an index value of 2, and so on. For perspective, the following input and output parameter values correspond to the following script tool UI. 

```R
  input_data <- in_params[[1]]
  train_percentage_size <- (in_params[[2]])/100
  dependent_variable <- in_params[[3]]
  independent_variables <- in_params[[4]]
  
  output_prediction_data <- out_params[[1]]
```

![image](image/Logistic_Regression_Script_Tool.png)

### Open and Read-in Data

**arcgisbinding** functions like `arc.open()`, `arc.select()`, and, if needed, `arc.data2sp()` or `arc.data2sf()` are then used to bring the data from your created geoprocessing script tool user interface into R and convert it into the needed format to perform your desired R functions on it. 

Here is one particular example. In this case, both the full data set and a subset of the data set are brought in for use in the script. What you choose to do will be based on the requirements of your R functions. 

```R
  d <- arc.open(input_data)
  fields_list <- append(c(dependent_variable), independent_variables)
  d_df_full <- arc.select(d)
  d_df <- arc.select(d, fields = fields_list)
```

### Custom R Functionality

So far, we've covered pieces from within the `tool_exec()` function that will likely be ubiqutous to all scripts wrapping R functionality. Once your desired data is inside R, what happens next and the resulting output of your tool is not limited and can be as creative as your coding. Any R function that works with data frames, spatial data frames, or rasters can be used. Additionally, any R diagnostic measures, be they statistical measures or charts, can either be printed out in your tool's messages window or produced by R when you run your tool. 

### Generate Results and Messages

To generate results from R functions for your tool's messages, you can make use of print statements. Here is one particular example of how this can be done. 

```R
  arc.progress_label("Running diagnostics on fitted model...")
  arc.progress_pos(80)
  
  #Summary of model fit
  cat(paste0("\n", "............................................", "\n"))
  cat(paste0("\n", "............................................", "\n"))
  cat(paste0("\n"))
  cat(paste0("\n", "Summary of Fitted Logistic Regression Model", "\n"))
  cat(paste0("\n", "............................................", "\n"))
  cat(paste0("\n"))
  print(summary(d_df_train.log))
  
  #Hosmer-Lemeshow Test
  cat(paste0("\n", "............................................", "\n"))
  cat(paste0("\n", "............................................", "\n"))
  cat(paste0("\n"))
  cat(paste0("\n", "Hosmer-Lemeshow Goodness of Fit Test Results", "\n"))
  cat(paste0("\n", "............................................", "\n"))
  cat(paste0("\n"))
  HL <- HLgof.test(fit = fitted(d_df_train.log), obs = d_df_train$Seagrass)
  print(HL)
```

Additionally, R charts can be used to further communicate results from your analysis in R. To have a chart from R pop-up when you or others run your tool, you will simply make a call to your designated chart function of choice. 

Here is an example of producing an ROC curve when running a script tool designed to perform logistic regression. The R `plot()` call is all you need for this chart to appear when someone else runs your script tool from within ArcGIS. 

```R
  #ROC Curve
  d_df_test.log.pred <- predict(d_df_train.log, newdata = d_df_test, type = 'response')
  pred <- prediction(d_df_test.log.pred, d_df_test$Seagrass)
  perf <- performance(pred, measure = "tpr", x.measure = "fpr")
  plot(perf)
```

Finally, to return any results from R to ArcGIS, you can use the `arc.write()` function. For full documentation on the `arc.write()` function, see the Jupyter Notebook called R-bridge-reading-converting-writing-data. 

```R
  arc.progress_label("Writing output...")
  arc.progress_pos(80)
  
  if(!is.null(output_prediction_data) && output_prediction_data != "NA")
    arc.write(output_prediction_data, d_df_full, shape_info = arc.shapeinfo(d))
```

### Example R Script Tools

Examples of R script tool can be found on the [GitHub project associated with the R-ArcGIS bridge](https://github.com/R-ArcGIS/r-sample-tools). Each one of the sample script tools comes with some associated data and documentation. These script tools can also be used as templates for your own script tool creation. Feel free to download them and to modify them as needed. If you create a script tool you would like to share with the community, we would love to help you host it. For reference, check out some of the tools created by our users, including [CHANS-tools](https://github.com/R-ArcGIS/CHANS-tools). 

## Components of ArcGIS Script Tool UI Creation

The customization of the user interface of your tool is constructed by the tool properties of the tool within the ArcGIS toolbox, full details of which can be found [here](http://pro.arcgis.com/en/pro-app/help/analysis/geoprocessing/basics/create-a-python-script-tool.htm). 

Through these properties, you have complete control over how users interact with your tool and its options. You can customize the style and type of parameter box and the allowed options available for each. Parameter defaults, ranges, and drop-downs can all be tailored to the tool and the user experience you are designing. 

## Hands-on Practice: Script Tool Creation

To practice your understanding of how to create an R script tool, we will be using a modified version of the web course: [Integrate R Scripts into ArcGIS Geoprocessing Tools](https://www.esri.com/training/catalog/58b5e578b89b7e000d8bfffd/integrating-r-scripts-into-arcgis-geoprocessing-tools/). We have made adjustments to the original content to reflect the **arcgisbinding** package's new functions for raster data. To access the original course, you simply need to sign into your Esri account. As long as you have a current maintenance subscription, you have full access to the course and all of its content. If you have forgotten your Esri account username or password, you can recover it via email. 

![image](image/Course_Sign_In.png)

Practice your understanding of script tools using the challenge below. 

### Exercise: Create an R script geoprocessing tool

You work as a GIS specialist for an environmental organization in Portland, Oregon. The organization has been awarded a large contract from the National Park Service to determine areas in several national parks that have concentrations of noxious weeds.

You have been provided with an ArcGIS dataset containing sampling locations. You will create a geoprocessing tool to determine areas in which the weeds are highly clustered. In this exercise, you will start the initial phases of the contract by implementing the code to perform the analysis, creating a script tool referencing the R code, and executing the tool.

![image](image/Script_Tool_Exercise.png)

#### Step 1: Access the data

The data for this exercise can be found in the following location on the laptop provided for this preconference seminar: `C:/data/data.gdb`

#### Step 2: Open the script in R or RStudio

In this step, you will open the script in R or RStudio for editing.

 * Open R or RStudio, depending on your preference.
 * From the File menu, click Open File.
 * In the Open File dialog box, browse to and open the `C:\data\ScanStatistic.R` script.

The ScanStatistic R script is not complete. Comments in the script indicate the steps where you will add code.

> Note: A completed script, ScanStatistic_Solution.R, will be provided for reference towards the end of our hands-on session.

#### Step 3: Implement code for obtaining environmental parameters

In this step, you will create code to define variables representing your script tool input and output parameters and their values provided by the users of your tool. 

 * Locate Step 1 in the script: Define input and output parameters.
 * Create variables in the script for the following input parameter values.
 
![image](image/Input_Parameters4.png)
 
 * Create variables for the following output parameter values.
 
![image](image/Output_Parameters2.png)
 
#### Step 4: Implement the progress bar

In this step, you will implement code to label and accurately report the current state of the process in the geoprocessing tool progress bar.

 * Locate Step 2 in the script: Create a progress label.
 * Implement the code for a progress label that will display "Loading data..." in the geoprocessing window.

Like other progress bars, the R-ArcGIS bridge progress bar displays the label when the script progress has reached the percentage specified.

 * Implement the code for displaying the label when the progress has reached 40 percent.
 * Save your modified file.
 
#### Step 5: Create a new script tool

In this step, you will create a new toolbox in ArcGIS Pro and add the ScanStatistic R script to the toolbox as a new script tool.

 * Open ArcGIS Pro.
 * Under Create A New Project, click Blank.
 * In the Create a New Project dialog box, name your project MyRProject.
 * For Location, browse to `C:\data`, and save your project to that folder.
 * Once your project is open, on the Insert tab, click the Toolbox drop-down list, and select New Toolbox to create a new toolbox.

 ![image](image/Add_Toolbox.png)
 
 * In the New Toolbox dialog box, browse to `C:\data`.
 * For Name, type Scan Statistic Tools, and then click Save.
 * In the Catalog pane, expand Toolboxes to see the Scan Statistic Tools toolbox that you have just created.
 * Right-click the Scan Statistic Tools toolbox, point to New, and select Script to create a script in the toolbox.
 
The dialog box for the tool opens. This box is where you will configure the properties.

#### Step 6: Configure the tool

In this step, you will configure the script as a tool in the toolbox.

 * In the dialog box, for Name, type ScanStatistic.
 * For Label, type Scan Statistic.

The Label property defines how the tool label will be displayed in the toolbox.

 * For Script File, browse to `C:\data`, and click the ScanStatistic.R file.
 * Click OK.

The ScanStatistic.R file is the script that the tool will use. Next, you will define the tool parameters.

 * In the dialog box, click Parameters.
 * Type the following parameter values line by line. Leave blank or missing parameter values empty.
 
 ![image](image/Script_Parameters.png)
 
Filters allow predefined values to be configured for individual tool parameters. Setting predefined values for tool parameters  can help ensure consistency or limit the number of possible entries.

Next, in the Filter column, you will configure a value list for the Agency tool parameter that contains specified values.

 * In the table, for the Agency tool parameter, under Filter, click the box to activate it.
 * In the drop-down list, select Value List.
 * In the Value List Filter dialog box, type the following values, adding a new item line when necessary:
  * Agency = 'Canyonlands N. P.'
  * Agency = 'Dinosaur N. M.'
  * Agency = 'Zion N. P.'
  
  ![image](image/Value_List.png)
  
 * Click OK.
 * For the Model Type parameter, under Filter, use the same process to create a value list filter containing the following parameter values:
  * Poisson
  * Binomial
  
  ![image](image/Statistic_Type.png)
  
 * For the Beginning Distance tool parameter, under Filter, select Range.
 * In the Range dialog box, create a range filter with the following values:
  * Minimum: 0.000001
  * Maximum: 1000000
  
  ![image](image/Range.png)
    
 * For the Simulations tool parameter, under Filter, create a value list with the following values:
  * 10
  * 50
  * 100
  * 500
  
  ![image](image/Value_List2.png)
  
After you have configured all the tool parameters required for this script to execute successfully, you should see nine configured parameters displayed in the dialog box.

 * Click OK to save.
 
  ![image](image/Scan_Statistic.png)
 
#### Step 7: Run the tool

In this step, you will run the script tool to determine clustered locations for noxious weeds in Zion National Park.

 * On the Insert tab, in the Project group, click New Map.

Now you will add the new data to the map.

 * On the Map tab, in the Layer group, click the Add Data down arrow.
 * In the Add Data dialog box, browse to `C:\data\data.gdb`, click the NoxiousWeeds feature class, and click OK to add it to the map.
 * In the Catalog pane, if necessary, expand Toolboxes, and then expand Scan Statistic Tools.tbx.
 * Double-click the Scan Statistic tool Script icon.
 * In the Geoprocessing pane, for Scan Statistic, click Parameters, and set the following parameters:
  * Occurrence Dataset: 
    - Browse to ..`C:\data\data.gdb` and select NoxiousWeeds
    - Click OK.
  * Agency: Agency='Zion N. P.'
  * Model Type: Poisson
  * Number Of Distance Bands: 6
  * Beginning Distance: 0.4
  * Maximum Distance: 600
  * Simulations: 100
  * Output Raster:
    - Browse to `C:\data\data.gdb`
    - For name, type Scan_Statistic_LRTS.tif.
  * Click Save.
  * Output Feature Class:
    - Browse to `C:\data\data.gdb`
    - For Name, type Zion_Subset.
 * Click Run to execute the Scan Statistic tool, and observe the progress bar while the tool is running.

The progress bar reflects the status of the execution process. Because the tool generates raster and feature class results, the tool may take a few minutes to complete.
